## Dependent and Independent State Transition Assumptions

Consider the case when there are two sources of decrement fro a given state in a time step, e.g.:

  * let $q$ denote the probability of death
  * let $r$ denote the probability of lapse
 

Let's assume that both assumptions have been derived independently. This is, e.g., the case when 
during the derivation of these transition assumptions the other cause of decrement was eliminated 
by considering the expsure time for one decrement only.

In case the assumptions can not directly used as state transition assumptions by ```PyProtolinc```.

In [24]:
import math
import numpy as np

# let's set the lapse rate to 50% and the death rate to 1%
q = 0.01
r = 0.5

It is intuitively clear that due to the high probabilty for lapses there are effectively less than 1% death cases
when considering both forces at work. More specificall we can argue that on average there are only ~75% exposed to death so 
that the effective death rate should be somewhere near 0.0075. We look at the equations in the continuous model:

$$
\begin{array}{ccc}
l(t)    &=& \int_0^t(f_q(s) + f_r(s))l(s)ds\\
l_0q^d(t) &=& \int_0^tf_q(s)l(s)ds\\
l_0r^r(t) &=& \int_0^tf_r(s)l(s)ds
\end{array} 
$$

Here the *forces of decrement* $f_q$ and $f_r$ from the independent decrements have been incorporated and $q^d$ and $r^d$ respectively denote the
*dependent* transition assumptions. Under the *additional assumption* that these
forces are constant between times $0$ and $1$ we can then solve the equations to find first:

$$
l(t) = l_0\exp(-(f_q+f_r)t)
$$

and based on this
$$
\begin{array}{ccccc}
q^d(t) &=& f_q\int_0^t\exp(-(f_q+f_r)s)ds &=& -\frac{f_q}{f_q+f_r}p(s)|^t_0 &=& \frac{f_q}{f_q+f_r}(1 - p(t))\\
r^d(t) &=& f_r\int_0^t\exp(-(f_q+f_r)s)ds &=& -\frac{f_r}{f_q+f_r}p(s)|^t_0 &=& \frac{f_r}{f_q+f_r}(1 - p(t))
\end{array} 
$$


Note that on the other hand we have the equation

$$
q = q^i(1) = f_q\int_0^1\exp(-f_qs)ds = -\exp(-f_qs)|^1_0 = 1 - \exp(-f_q)
$$

for the independent assumptions $q^i$ (and similar for $r^i$) when assuming *constant force for this case as well*.

Consequently $f_q = - \ln(1-q)$ and analogously $f_r = - \ln(1-r)$

For the numerical example this implies

In [25]:
f_q = -math.log(1 - q)
f_r = -math.log(1 - r)
f_q, f_r

(0.01005033585350145, 0.6931471805599453)

This translates to the dependent transition assumptions on the yearly grid as follows:

In [26]:
f = f_q + f_r
p = math.exp(-f)
q_d = f_q * (1 - p) / f
r_d = f_r * (1 - p) / f
q_d, r_d

(0.007217630164430397, 0.4977823698355696)

As expected on can see a small reduction of the lapse rate and a significant adjustment (in relative terms) for
the mortality when comparing with the independent rates (and the value of 0.0072 is close to what we have expected).

The formulae obtained can also directly by used to convert independent assumptions to dependent assumptions valid for *fractions of the year*.

In [29]:
def mort_and_lapse_for_fractions(q, r, t):
    """ Return a 'dependent transition matrix' for a model with two independent
        decrements :q and :r for the time span t. """
    assert 0 <= t and t <= t
    
    f_q = -math.log(1 - q)
    f_r = -math.log(1 - r)
    f = f_q + f_r
    p_t = math.exp(-f * t)
    q_d = f_q * (1 - p_t) / f
    r_d = f_r * (1 - p_t) / f
    
    M_frac = np.array([[1 - q_d - r_d, q_d, r_d],
                       [0, 1, 0],
                       [0, 0, 1]])
    return M_frac

M4 = mort_and_lapse_for_fractions(q, r, 0.25)
M4

array([[0.83878624, 0.00230412, 0.15890963],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ]])

Applying this state transition matrix four times we get back what we have calculated above.

In [28]:
np.linalg.matrix_power(M4, 4)

array([[0.495     , 0.00721763, 0.49778237],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ]])

## Assumptions for Fractions of a Year

In sections the options for adjusting yearly assumptions for different timesteps are discussed. Let's start the discussion by looking at two options described in section 2.6 of Gerber's book *Life Insurance Mathematics*.

We start with a simple example including two states, one representing the active phase (or, e.g., the state of an annuity in payment) and a terminal state. Usually the the decrement assumption is then specified by a mortality rate `q` which described the probability of an individual to transition from *active* to *death*  in the course of one year.

A state transition matrix would then look as follows:

In [6]:
q = 0.02
M = np.array([[1 - q, q], [0, 1]])
M

array([[0.98, 0.02],
       [0.  , 1.  ]])

### Constant Force Transitions with One State Transition

In the case described in this section one considers the so-called *force of mortality* (denoted by $f$ below) to be constant.

To understand that consider the continous setup where the evolution of the *actives portfolio* would be noted by $l(t)$ and described by the equation

$$
1 - l(t) = \int_0^tf(s)l(s)ds
$$


In general $f$ is time dependent but if one assumes that $f$ is a constant, then we can differentiate to find

$$ -l'(t) = l(t)f$$

which implies that $l(t) = l_0\exp(-ft)$.

Considering that for $t=1$ we have $l(1) = l_0(1 - q)$ we can conclude that $f = -\ln(1-q)$.

Consequently we can now calculate *fractional transition matrices* by noting that for $t \in [0, 1]$ we have
$$ l(t) = l_0\exp(-ft) = l_0\exp(t\ln(1-q)) = l_0 (1-q)^t. $$

This translates into the following transition matrix for $t = 0.5$:

In [7]:
def m_t_const_force(q, t):
    return np.array([[(1 - q)**t, 1 - (1 - q)**t], [0, 1]])

M2 = m_t_const_force(q, 0.5)
M2

array([[0.98994949, 0.01005051],
       [0.        , 1.        ]])

We can multiply back to confirm the expected:

In [8]:
np.dot(M2, M2)

array([[0.98, 0.02],
       [0.  , 1.  ]])

And similarly

In [9]:
M12 = m_t_const_force(q, 1/12)
M12

array([[0.99831786, 0.00168214],
       [0.        , 1.        ]])

In [10]:
np.linalg.matrix_power(M12, 12)

array([[0.98, 0.02],
       [0.  , 1.  ]])

### Linearity of q

An alternative method is obtained when assuming that $_uq = uq$.

In [11]:
def m_t_linear(q, t):
    return np.array([[1 - q*t, q*t], [0, 1]])

L2 = m_t_linear(q, 0.5)
L2

array([[0.99, 0.01],
       [0.  , 1.  ]])

In [12]:
np.dot(L2, L2)

array([[0.9801, 0.0199],
       [0.    , 1.    ]])

In this case the calculation is simpler but not exact.

In [13]:
L12 = m_t_linear(q, 1/12)
L12

array([[0.99833333, 0.00166667],
       [0.        , 1.        ]])

In [14]:
np.linalg.matrix_power(L12, 12)

array([[0.98018232, 0.01981768],
       [0.        , 1.        ]])

This problem becomes more severe for higher values of $q$, for example we take q = 1:

In [15]:
L2 = m_t_linear(0.9, 0.5)
np.dot(L2, L2)

array([[0.3025, 0.6975],
       [0.    , 1.    ]])

We see that more than 30% survive while there should be only 10%. Note that this works as expected for the constant force method.

In [16]:
L2 = m_t_const_force(0.9, 0.5)
np.dot(L2, L2)

array([[0.1, 0.9],
       [0. , 1. ]])

### Constant Force Transitions with Two Decrements

Next we look at the case with one active state and two decrements. A transition matrix on the yearly grid would look like this whereby $q$ and $r$ represent the **dependent** yearly transition probabilities.

$$
  \left[ {\begin{array}{ccc}
    1 - q - r & q & r \\
    0         & 1 & 0 \\
    0         & 0 & 1 \\
  \end{array} } \right]
$$

We proceed as before by writing down the continuous transition equations
assuming *constant force of decrements* for both transitions:

$$
\begin{array}{ccc}
l_0q(t) &=& \int_0^tf_ql(s)ds\\
l_0r(t) &=& \int_0^tf_rl(s)ds
\end{array} 
$$

Adding the equations and noting that $l(t) = l_0(1 - q(t) - r(t))$ we find similar to before that 

$$
1 - l(t) = \int_0^t(f_q+f_r)l(s)ds
$$

and consequently $l(t) = l_0\exp(-(f_q+f_r)t)$ and $f_q + f_r = -\ln(1-q-r).$

Using the representation of $l(t)$ in the expression for $q(t)$ one obtains

$$
\begin{array}{ccc}
l_0q(t) &=& \int_0^tf_ql(s)ds\\
        &=& f_q \int_0^tl_0\exp(-(f_q+f_r)t)ds\\
        &=& f_q \int_0^tl_0\exp(-(f_q+f_r)t)ds\\
        &=& -\frac{f_q}{f_q+f_r}l(t)|^t_0.
\end{array}
$$

Consequently
$$
\begin{array}{ccc}
q(t) &=& \frac{f_q}{f_q+f_r}(1 - p(t)), \\
r(t) &=& \frac{f_r}{f_q+f_r}(1 - p(t)),
\end{array} 
$$

Noting that for $t=1$ we have $q(t)=q$, $r(t)=r$ and $1 - p(1) = q + r$, so that we find
$$
\begin{array}{ccc}
q &=& \frac{f_q}{f_q+f_r}(q + r), \\
r &=& \frac{f_r}{f_q+f_r}(q + r). \\
\end{array} 
$$
We can now easily obtain the forces from this:
$$
\begin{array}{cccc}
f_q &=& \frac{q}{q+r}(f_q+f_r) = -\ln(1-q-r)\frac{q}{q+r}, \\
f_r &=& \frac{r}{q+r}(f_q+f_r) = -\ln(1-q-r)\frac{r}{q+r}. \\
\end{array} 
$$

We continue with a numerical example:

In [17]:
q = 0.1
r = 0.01
M = np.array([[1 - q - r, q, r],
              [0, 1, 0],
              [0, 0, 1]])
M

array([[0.89, 0.1 , 0.01],
       [0.  , 1.  , 0.  ],
       [0.  , 0.  , 1.  ]])

In [18]:

def fractional_transition(q, r, t):
    """ The function shall return a fractional transition matrix :M for the duration :t."""
    
    f_qr = -math.log(1-q-r)
    
    f_q = f_qr * q / (q + r)
    f_r = f_qr * r / (q + r)
    
    p_t = math.exp(-f_qr * t)   # (1 - q -r)**t
    
    q_t = (1 - p_t) * f_q / f_qr
    r_t = (1 - p_t) * f_r / f_qr
    
    M_frac = np.array([[1 - q_t - r_t, q_t, r_t],
                       [0, 1, 0],
                       [0, 0, 1]])
    return M_frac
    
M2 = fractional_transition(q, r, 0.5)
M2

array([[0.94339811, 0.05145626, 0.00514563],
       [0.        , 1.        , 0.        ],
       [0.        , 0.        , 1.        ]])

We check that we get the original matrix back:

In [19]:
np.dot(M2, M2)

array([[0.89, 0.1 , 0.01],
       [0.  , 1.  , 0.  ],
       [0.  , 0.  , 1.  ]])

We repeat the check for the monthly transitions:

In [20]:
M12 = fractional_transition(q, r, 1 / 12)
M12

array([[9.90335850e-01, 8.78559127e-03, 8.78559127e-04],
       [0.00000000e+00, 1.00000000e+00, 0.00000000e+00],
       [0.00000000e+00, 0.00000000e+00, 1.00000000e+00]])

In [21]:
np.linalg.matrix_power(M12, 12)

array([[0.89, 0.1 , 0.01],
       [0.  , 1.  , 0.  ],
       [0.  , 0.  , 1.  ]])